In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# データ結合

In [2]:
import os

# データセットのファイル構造を定義
dataset_parts = {
    'p5': ['p5a', 'p5b'],
    'p11': ['p11ab', 'p11c'],
    'p16': ['p16abc', 'p16d'],
    'p21': ['p21abcd', 'p21e']
}

# 通常のデータセットはp1からp29まで
all_datasets = [f'p{i}' for i in range(1, 30)]

# 特殊処理をするデータセットを除外
regular_datasets = [d for d in all_datasets if d not in dataset_parts]

# データを格納する辞書
data_dict = {}

# 通常のデータを処理
for dataset in regular_datasets:
    part_files = [f'../Data_JPSC/JPSC_general/data/{dataset}/{dataset}_{i}.csv' for i in range(1, 4)]
    merged_data = pd.read_csv(part_files[0])
    for part in part_files[1:]:
        merged_data = pd.merge(merged_data, pd.read_csv(part), on=['ID', 'PANEL'], how='inner')
    data_dict[dataset] = merged_data

# 特殊な構造を持つデータを処理
for key, parts in dataset_parts.items():
    part_dataframes = []
    for part in parts:
        part_files = [f'../Data_JPSC/JPSC_general/data/{part}/{part}_1.csv',
                      f'../Data_JPSC/JPSC_general/data/{part}/{part}_2.csv',
                      f'../Data_JPSC/JPSC_general/data/{part}/{part}_3.csv']
        merged_part_data = pd.read_csv(part_files[0])
        for part_file in part_files[1:]:
            merged_part_data = pd.merge(merged_part_data, pd.read_csv(part_file), on=['ID', 'PANEL'], how='inner')
        part_dataframes.append(merged_part_data)
    # p5, p11, p16, p21の2つのパートを結合
    data_dict[key] = pd.concat(part_dataframes, axis=0)

# 最後に全てのデータを結合
data = pd.concat(data_dict.values(), axis=0)
data = data.drop(columns=['REC','REC_x','REC_y'])
data = data.sort_values(by=['ID','PANEL'])
data.reset_index(drop=True, inplace=True)

# データフレームの出力
data


,ID,PANEL,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,...,P186D,P186E,P186F,P186G,P186H,P186I,P186J,P186K,P186L,P186M
0,1,1,2,3,2,99999,2,0,2,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,2,3,2,99999,2,0,2,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3,1,3,2,99999,2,0,2,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,1,3,2,99999,3,0,2,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,1,3,2,99999,6,0,2,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53978,5648,25,2,8,2,99999,2,0,2,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53979,5648,26,2,8,2,99999,2,0,2,31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53980,5648,27,2,8,2,99999,2,0,2,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53981,5648,28,2,8,2,99999,2,0,2,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data_original = data.copy()

# 欠損値処理

## 第1子が生まれたか

## 子どもの人数

In [4]:
data['Q57'] = data['Q57'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 8 else x)

## 収入

In [5]:
# 3Q7 問７．あなたの仕事の給与は、時給計算ですか、日給計算ですか。
# 妻の給料

# 月給 19662 non-null
data['Q149'] = data['Q149'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 9998 else x)
#万円単位に変換
data['Q149'] = data['Q149'].apply(lambda x: x/10)
# 日給 1470 non-null
data['Q150'] = data['Q150'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 99998 else x)
# 時給 13509 non-null
data['Q151'] = data['Q151'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 99998 else x)


# 4Q7 問７．ご主人のお仕事の給与は、時給計算ですか、日給計算ですか。
# 夫の給料

# 月給 28678 non-null
data['Q220'] = data['Q220'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 9998 else x)
#万円単位に変換
data['Q220'] = data['Q220'].apply(lambda x: x/10)
# 日給 2155 non-null
data['Q221'] = data['Q221'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 99998 else x)
# 時給 498 non-null
data['Q222'] = data['Q222'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 99998 else x)

### 月収

In [6]:
#6Q9．この９月の手取り収入（収入総額から税金、社会保険料などを差し引いた額）はどのくらいですか。（ボーナス、臨時の給与は除きます）
#夫の手取り
data['Q307B'] = data['Q307B'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 998 else x)
#妻の手取り
data['Q308B'] = data['Q308B'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 998 else x)

In [7]:
#夫の年収
Q296 = ['Q296A', 'Q296B', 'Q296C', 'Q296D', 'Q296E', 'Q296F']
for column in Q296:
    data[column] = data[column].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 9998 else x)

#妻の年収
Q297 = ['Q297A', 'Q297B', 'Q297C', 'Q297D', 'Q297E', 'Q297F']
for column in Q297:
    data[column] = data[column].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 9998 else x)


In [8]:
data['HH_Total_Inc'] = data['Q296F'] + data['Q297F']

C:\Users\ouhsj\AppData\Local\Temp\ipykernel_22940\3232106426.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['HH_Total_Inc'] = data['Q296F'] + data['Q297F']


## 9月の月収を推定するコード

In [9]:
# 夫の収入を推定する関数
def estimate_husband_income(row):
    # Q307Bが欠損値でなければその値を使用
    if not pd.isna(row["Q307B"]):
        return row["Q307B"]
    
    # Q307Bが欠損値の場合、Q220を使用
    elif not pd.isna(row["Q220"]):
        return row["Q220"]
    
    # Q220も欠損値の場合、Q296A / 12 を使用
    elif not pd.isna(row["Q296A"]):
        return row["Q296A"] / 12
    
    # Q296Aも欠損値の場合、Q296B / 12 を使用
    elif not pd.isna(row["Q296B"]):
        return row["Q296B"] / 12
    
    # すべてが欠損値の場合は np.nan を返す
    return np.nan

# 妻の収入を推定する関数
def estimate_wife_income(row):
    # Q308Bが欠損値でなければその値を使用
    if not pd.isna(row["Q308B"]):
        return row["Q308B"]
    
    # Q308Bが欠損値の場合、Q149を使用
    elif not pd.isna(row["Q149"]):
        return row["Q149"]
    
    # Q149も欠損値の場合、Q297A / 12 を使用
    elif not pd.isna(row["Q297A"]):
        return row["Q297A"] / 12
    
    # Q297Aも欠損値の場合、Q297B / 12 を使用
    elif not pd.isna(row["Q297B"]):
        return row["Q297B"] / 12
    
    # すべてが欠損値の場合は np.nan を返す
    return np.nan

# データフレームに対して適用する
data["H_Inc_Sept"] = data.apply(estimate_husband_income, axis=1)
data["W_Inc_Sept"] = data.apply(estimate_wife_income, axis=1)


## 時間配分

In [10]:
## 労働時間
data['Q152'] = data['Q152'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 11 else x)

In [11]:
hours = [
    "Q493AH", "Q493BH", "Q493CH", "Q493DH", "Q493IH", "Q493MH", #妻 平日
    "Q494AH", "Q494BH", "Q494CH", "Q494DH", "Q494IH", "Q494MH", #妻 休日
    "Q495AH", "Q495BH", "Q495CH", "Q495DH", "Q495IH", "Q495MH", #夫 平日
    "Q496AH", "Q496BH", "Q496CH", "Q496DH", "Q496IH", "Q496MH"  #夫 休日
    ]
minutes = [
    "Q493AM", "Q493BM", "Q493CM", "Q493DM", "Q493IM", "Q493MM", #妻 平日
    "Q494AM", "Q494BM", "Q494CM", "Q494DM", "Q494IM", "Q494MM", #妻 休日
    "Q495AM", "Q495BM", "Q495CM", "Q495DM", "Q495IM", "Q495MM", #夫 平日
    "Q496AH", "Q496BM", "Q496CH", "Q496DM", "Q496IM", "Q496MM" #夫 休日
    ]

for column in hours:
    data[column] = data[column].apply(lambda x: np.nan if isinstance(x, (int, float)) and x > 24 else x)
    data[column] = data[column].apply(lambda x: x*60)

for column in minutes:
    data[column] = data[column].apply(lambda x: np.nan if isinstance(x, (int, float)) and x > 6 else x)
    data[column] = data[column].apply(lambda x: x*10)

# Create new columns based on hours and minutes
# 単位は時間
for hour_col, min_col in zip(hours, minutes):
    new_col = hour_col[:-1]  # New column name, e.g., Q493A
    data[new_col] = data[hour_col] + data[min_col]
    data[new_col] = data[new_col].apply(lambda x: x/60)
    data[new_col] = data[new_col].apply(lambda x: np.nan if isinstance(x, (int, float)) and x > 24 else x)

## 家事の割合

In [12]:
# 問29．炊事・洗濯・掃除などの家事について、ご夫婦でどの程度分担しあっていますか。全体を100％としたときの、それぞれの貢献分をお答えください。
data['Q1233A'] = data['Q1233A'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 998 else x)
data['Q1233B'] = data['Q1233A'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 998 else x)

## 支出

In [13]:
# 置き換える対象の列
columns_6Q1 = [
    'Q293A', 'Q293B', 'Q293C', 'Q293E', 'Q293F', 'Q293G', #支出
    'Q294A', 'Q294B', 'Q294C', 'Q294D', 'Q294E', 'Q294F',  #貯蓄
    'Q295' #ローン返済
]

for column in columns_6Q1:
    data[column] = data[column].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 998 else x)
    data[column] = data[column].apply(lambda x: x/10) #万円単位に換算

In [14]:
# 'Q293A'と'Q294A'を合計し、新しい列を作成
data['C_plus_S_Total'] = data['Q293A'] + data['Q294A']
data['C_plus_S_All'] = data['Q293B'] + data['Q294B']
data['C_plus_S_Wife'] = data['Q293C'] + data['Q294C']
data['C_plus_S_Husband'] = data['Q293E'] + data['Q294D']
data['C_plus_S_Child'] = data['Q293F'] + data['Q294E']
data['C_plus_S_Other'] = data['Q293G'] + data['Q294F']

data['C_plus_S_Public'] = data['C_plus_S_All'] + data['C_plus_S_Other']

# 比率を計算
data['C_plus_S_All_Ratio'] = data['C_plus_S_All'] / data['C_plus_S_Total']
data['C_plus_S_Wife_Ratio'] = data['C_plus_S_Wife']  / data['C_plus_S_Total']
data['C_plus_S_Husband_Ratio'] = data['C_plus_S_Husband'] / data['C_plus_S_Total']
data['C_plus_S_Child_Ratio'] = data['C_plus_S_Child'] / data['C_plus_S_Total']
data['C_plus_S_Other_Ratio'] = data['C_plus_S_Other'] / data['C_plus_S_Total']
data['C_plus_S_Public_Ratio'] = data['C_plus_S_Public'] / data['C_plus_S_Total']

C:\Users\ouhsj\AppData\Local\Temp\ipykernel_22940\382811819.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['C_plus_S_Other_Ratio'] = data['C_plus_S_Other'] / data['C_plus_S_Total']
C:\Users\ouhsj\AppData\Local\Temp\ipykernel_22940\382811819.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['C_plus_S_Public_Ratio'] = data['C_plus_S_Public'] / data['C_plus_S_Total']


In [15]:
data.rename(columns={'Q293A':'Cons_Total'}, inplace=True)
data.rename(columns={'Q293B':'Cons_All'}, inplace=True)
data.rename(columns={'Q293C':'Cons_Wife'},  inplace=True)
data.rename(columns={'Q293E':'Cons_Husband'}, inplace=True)
data.rename(columns={'Q293F':'Cons_Child'}, inplace=True)
data.rename(columns={'Q293G':'Cons_Other'}, inplace=True)

data.rename(columns={'Q294A':'Saving_Total'}, inplace=True)
data.rename(columns={'Q294B':'Saving_All'}, inplace=True)
data.rename(columns={'Q294C':'Saving_Wife'},  inplace=True)
data.rename(columns={'Q294D':'Saving_Husband'}, inplace=True)
data.rename(columns={'Q294E':'Saving_Child'}, inplace=True)
data.rename(columns={'Q294F':'Saving_Other'},   inplace=True)

data['Cons_All_Ratio'] = data['Cons_All'] / data['Cons_Total']
data['Cons_Wife_Ratio'] = data['Cons_Wife']  / data['Cons_Total']
data['Cons_Husband_Ratio'] = data['Cons_Husband'] / data['Cons_Total']
data['Cons_Child_Ratio'] = data['Cons_Child'] / data['Cons_Total']
data['Cons_Other_Ratio'] = data['Cons_Other'] / data['Cons_Total']

data['Cons_Public'] = data['Cons_All'] + data['Cons_Other']

data['Saving_All_Ratio'] = data['Saving_All'] / data['Saving_Total']
data['Saving_Wife_Ratio'] = data['Saving_Wife']  / data['Saving_Total']
data['Saving_Husband_Ratio'] = data['Saving_Husband'] / data['Saving_Total']
data['Saving_Child_Ratio'] = data['Saving_Child'] / data['Saving_Total']
data['Saving_Other_Ratio'] = data['Saving_Other'] / data['Saving_Total']

data['Saving_Public'] = data['Saving_All'] + data['Saving_Other']

data['Cons_Public_Ratio'] = data['Cons_Public'] / data['Cons_Total']
data['Saving_Public_Ratio'] = data['Saving_Public'] / data['Saving_Total']

data['Saving_Ratio'] = data['Saving_Total'] / (data['Cons_Total'] + data['Saving_Total'])

C:\Users\ouhsj\AppData\Local\Temp\ipykernel_22940\3938519938.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['Cons_All_Ratio'] = data['Cons_All'] / data['Cons_Total']
C:\Users\ouhsj\AppData\Local\Temp\ipykernel_22940\3938519938.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['Cons_Wife_Ratio'] = data['Cons_Wife']  / data['Cons_Total']
C:\Users\ouhsj\AppData\Local\Temp\ipykernel_22940\3938519938.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` man

# 家計の管理タイプ

In [16]:
#下のように整形するとWarningが出るので，処理を行う列に対してのみ処理を行う．
#data['Q283'] = data['Q283'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 19 else x)

#家計の管理タイプ
def func_HH_Type(x):
    if np.isnan(x):
        return np.nan  # NaNの場合
    elif 15 <= x <= 18 or x == 9 or 4 <= x <= 5:
        return 'Wife'  # 妻が管理
    elif 11 <= x <= 14 or 2 <= x <= 3:
        return 'Joint'  # 共同で管理
    elif 6 <= x <= 8:
        return 'Husband'  # 夫が管理
    elif x == 10:
        return 'Separate'  # 別々で管理
    elif x == 1:
        return 'No_Income'  # 夫婦共に収入なし
    else:
        return None  # 該当なしの場合

# Apply transformations to 'Q283' and 'HH_Type'
Q283_transformed = data['Q283'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 19 else x)
HH_Type = Q283_transformed.apply(func_HH_Type)

# Update the DataFrame using pd.concat()
data = pd.concat([data.drop(columns=['Q283']), Q283_transformed.rename('Q283'), HH_Type.rename('HH_Type')], axis=1)


## 「夫から妻」「妻から夫」に渡す金額

In [17]:
# 夫から妻に渡した額
data['Q271'] = data['Q271'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 998 else x)
data['Q278'] = data['Q278'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 998 else x)

# 妻から夫に渡した額
data['Q274'] = data['Q274'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 998 else x)

# 夫のお小遣い
data['Q284'] = data['Q284'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 998 else x)

data['Q271'] = data['Q271'].apply(lambda x: x/10)
data['Q274'] = data['Q274'].apply(lambda x: x/10)
data['Q278'] = data['Q278'].apply(lambda x: x/10)
data['Q284'] = data['Q284'].apply(lambda x: x/10)

In [18]:
# 初期設定
data["Transfer_Money"] = np.nan

# 関数を定義して条件に基づいて値を設定
def calculate_transfere_money(row):
    # 15 <= Q283 <= 18 または 4 <= Q283 <= 5 の場合
    if 15 <= row["Q283"] <= 18 or 4 <= row["Q283"] <= 5:
        return row["H_Inc_Sept"]
    
    # 2 <= Q283 <= 3 の場合
    elif 2 <= row["Q283"] <= 3:
        if pd.isna(row["Q271"]):  # Q271 が欠損値の場合
            return row["H_Inc_Sept"] * 0.5
        else:
            return row["Q271"]
    
    # 11 <= Q283 <= 14 の場合
    elif 11 <= row["Q283"] <= 14:
        if pd.isna(row["Q278"]):  # Q278 が欠損値の場合
            return row["H_Inc_Sept"] * 0.5
        else:
            return row["Q278"]

    # Q283 == 6 の場合
    elif row["Q283"] == 6:
        if pd.isna(row["Q274"]):  # Q274 が欠損値の場合
            return -row["W_Inc_Sept"] * 0.5
        else:
            return -(row["W_Inc_Sept"] - row["Q274"])

    # 7 <= Q283 <= 8 の場合
    elif 7 <= row["Q283"] <= 8:
            return -row["W_Inc_Sept"]
    
    # Q283 が 9, 1, 10 の場合は 0
    elif row["Q283"] in [9, 1, 10]:
        return 0
    
    # デフォルトでは np.nan
    return np.nan

# apply 関数を使って行ごとに計算
data["Transfer_Money"] = data.apply(calculate_transfere_money, axis=1)


C:\Users\ouhsj\AppData\Local\Temp\ipykernel_22940\4145279653.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["Transfer_Money"] = np.nan


In [19]:
# data["Transfered_Money"]の値に基づいて、1, 0, -1を返す新しい列を作成
data["Transfer_Type"] = np.sign(data["Transfer_Money"])
data["Transfer_Type"] = data["Transfer_Type"].map({1: "Husband_to_Wife", 0: "No_Transfer", -1: "Wife_to_Husband"})

In [20]:
# Q283が2から3の範囲の行において、Q271が欠損値である割合を計算
q283_range_1 = data[(data["Q283"] >= 2) & (data["Q283"] <= 3)]
missing_q271_ratio = q283_range_1["Q271"].isna().mean()

# Q283が11から14の範囲の行において、Q278が欠損値である割合を計算
q283_range_2 = data[(data["Q283"] >= 11) & (data["Q283"] <= 14)]
missing_q278_ratio = q283_range_2["Q278"].isna().mean()

missing_q271_ratio, missing_q278_ratio

(0.03477608206154616, 0.022288261515601784)

In [21]:
# お金の渡す金額が，夫の収入に占める割合
data['Transfer_Ratio'] = data['Transfer_Money'] / data['H_Inc_Sept']

## 就業状態

In [22]:
data['Q142'] = data['Q142'].replace({
    1: 'Working',
    2: 'Student',
    3: 'Housewife',
    4: 'Other'
})
data['Q142'] = data['Q142'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 5 else x)

data['Q1025'] = data['Q1025'].replace({
    1: 'Working',
    2: 'Working',
    3: 'Student',
    4: 'Housewife',
    5: 'Other'
})
data['Q1025'] = data['Q1025'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 6 else x)

# 'Q142' 列の値を優先し、'Q1025' の値が欠損値でない場合は 'Q1025' を使用
data['Working_Status'] = data['Q142'].fillna(data['Q1025'])

In [23]:
data['Q147'] = data['Q147'].replace({
    1: 'Regular',
    2: 'Part-time',
    3: 'Other'
})
data['Q147'] = data['Q147'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 4 else x)

data['Q1112'] = data['Q1112'].replace({
    1: 'Regular',
    2: 'Dispatch',
    3: 'Fixed-term',
    4: 'Part-time',
    5: 'Other'
})
data['Q1112'] = data['Q1112'].apply(lambda x: np.nan if isinstance(x, (int, float)) and x >= 6 else x)

# 'Q142' 列の値を優先し、'Q1025' の値が欠損値でない場合は 'Q1025' を使用
data['Contract_Type'] = data['Q1112'].fillna(data['Q147'])

# 変数名の変更

In [24]:
data.rename(columns={'Q57': 'N_child'}, inplace=True)
data["log_N_child"] = np.log1p(data["N_child"]+ 1)

data.rename(columns={'Q493A':'W_Commute_Time'}, inplace=True)
data.rename(columns={'Q495A':'H_Commute_Time'}, inplace=True)
data.rename(columns={'Q493B':'W_Work_Time'}, inplace=True)
data.rename(columns={'Q495B':'H_Work_Time'}, inplace=True)
data.rename(columns={'Q493C':'W_Study_Time'}, inplace=True)
data.rename(columns={'Q495C':'H_Study_Time'}, inplace=True)
data.rename(columns={'Q493D':'W_Chore_ChildCare_Time'}, inplace=True)
data.rename(columns={'Q495D':'H_Chore_ChildCare_Time'}, inplace=True)
data.rename(columns={'Q493I':'W_Hobby_Time'}, inplace=True)
data.rename(columns={'Q495I':'H_Hobby_Time'}, inplace=True)
data.rename(columns={'Q493M':'W_Sleep_Time'}, inplace=True)
data.rename(columns={'Q495M':'H_Sleep_Time'}, inplace=True)


#---------------------------------------------------
data['W_Market_Time'] = data['W_Commute_Time'] + data['W_Work_Time']
data['H_Market_Time'] = data['H_Commute_Time'] + data['H_Work_Time']

data['W_Leisure_Time'] = data['W_Hobby_Time'] + data['W_Sleep_Time']
data['H_Leisure_Time'] = data['H_Hobby_Time'] + data['H_Sleep_Time']

data['W_Total_Time'] = data['W_Market_Time'] + data['W_Study_Time'] + data['W_Chore_ChildCare_Time'] + data['W_Leisure_Time']
data['H_Total_Time'] = data['H_Market_Time'] + data['H_Study_Time'] + data['H_Chore_ChildCare_Time'] + data['H_Leisure_Time']

In [25]:
data.rename(columns={'Q1233A':'H_Chore_Ratio'}, inplace=True)
data.rename(columns={'Q1233B':'W_Chore_Ratio'}, inplace=True)

# データ書き出し

In [26]:
# データの容量を圧縮
# 元の列名と現在の列名を比較
original_columns = set(data_original.columns)
current_columns = set(data.columns)

# 元のデータフレームにない新しい列を抽出
new_columns = current_columns - original_columns

# 常に保持する列 (IDとPANEL) を追加
columns_to_keep = ['ID', 'PANEL'] + list(new_columns)

# 新しい列だけを含むデータフレームを作成
data_to_csv = data[list(columns_to_keep)]

In [27]:
# データ書き出し
data_to_csv.to_csv('../Data_JPSC/data.csv', index=False)